In [33]:
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
from sklearn.utils import shuffle
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

data = pd.read_excel(r"/kaggle/input/new-data-ji-ou/ji ou  big126 .xlsx",engine="openpyxl")
X = data.iloc[:, :5].values
y0 = data.iloc[:, -1].values
y = data.iloc[:, -1].values

test_losses = []
temp = 100000.0
temp= torch.tensor(temp, dtype=torch.float32)
test_losses.append(temp)
print(torch.sqrt(sum(test_losses)))

tensor(316.2278)


In [34]:
input_size = X[0].shape[0]
output_size = 1
learning_rate = 1e-2
num_epochs = 100
jiaocha_numbers = 10
save_rate = 0.95
hid_feature = 32
alpha,beta = 0.5,0.5
num_while=0
#alpha+beta = 1 !!!

num1=8
num2=16

In [35]:
def KlLoss(mu,logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

class MLP(nn.Module):
    def __init__(self, input_size,hid_feature):
        super().__init__()
        self.input_size = input_size
        self.hid_feature = hid_feature

        self.encoder = nn.Sequential(
            nn.Linear(input_size,num1),
            nn.ELU(),
            nn.BatchNorm1d(num1),
            nn.Linear(num1,num2),
            nn.ELU(),
            nn.Linear(num2,hid_feature),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(hid_feature,num2),
            nn.ReLU(),
            nn.BatchNorm1d(num2),
            nn.Linear(num2,num1),
            nn.ELU(),
            nn.Linear(num1,input_size),
        )
        
        self.predicter = nn.Sequential(
            nn.Linear(hid_feature,num2),
            nn.ReLU(),
            nn.BatchNorm1d(num2),
            nn.Linear(num2,num1),
            nn.ELU(),
            nn.BatchNorm1d(num1),
            nn.Linear(num1,1),
        )
    
    def reparameterize(self,hid):
        mu,logvar = hid[:,hid_feature//2:],hid[:,:hid_feature//2]
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std
        return mu,logvar,z

    def forward(self, x):
        hid = self.encoder(x)
        out = self.decoder(hid)
        y = self.predicter(hid)
        return out,y
    

In [ ]:
while (torch.sqrt(sum(test_losses)/len(y)) > 0.32):
    num_while +=1
    test_losses = []
    
    def KlLoss(mu,logvar):
        return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    class MLP(nn.Module):
        def __init__(self, input_size,hid_feature):
            super().__init__()
            self.input_size = input_size
            self.hid_feature = hid_feature

            self.encoder = nn.Sequential(
                nn.Linear(input_size,num1),
                nn.ELU(),
                nn.BatchNorm1d(num1),
                nn.Linear(num1,num2),
                nn.ELU(),
                nn.Linear(num2,hid_feature),
            )

            self.decoder = nn.Sequential(
                nn.Linear(hid_feature,num2),
                nn.ReLU(),
                nn.BatchNorm1d(num2),
                nn.Linear(num2,num1),
                nn.ELU(),
                nn.Linear(num1,input_size),
            )

            self.predicter = nn.Sequential(
                nn.Linear(hid_feature,num2),
                nn.ReLU(),
                nn.BatchNorm1d(num2),
                nn.Linear(num2,num1),
                nn.ELU(),
                nn.BatchNorm1d(num1),
                nn.Linear(num1,1),
            )

        def reparameterize(self,hid):
            mu,logvar = hid[:,hid_feature//2:],hid[:,:hid_feature//2]
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            z = mu + eps*std
            return mu,logvar,z

        def forward(self, x):
            hid = self.encoder(x)
            out = self.decoder(hid)
            y = self.predicter(hid)
            return out,y

    
    kf = KFold(n_splits=jiaocha_numbers ,shuffle=True)
    results_df = pd.DataFrame()
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        model = MLP(input_size,hid_feature)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=0.1)
        lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epochs,last_epoch=-1)
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y0[train_index], y0[test_index]
        scaler = MinMaxScaler()
        
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
        
        train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        batch_size = X_train.shape[0]
        train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        
        losses = []
        
#        bar = tqdm(range(num_epochs))
#        bar.set_description("[ Training ]")
        bar = range(num_epochs)
        best = 100
        for idx,epoch in enumerate(bar):
            for i, (inputs, labels) in enumerate(train_loader):
                labels = labels 
                optimizer.zero_grad()
                out,y = model(inputs)
                label_loss = criterion(y, labels.unsqueeze(1))
                recon_loss = criterion(out, inputs)
                loss = alpha*recon_loss + beta*label_loss
                if  idx>num_epochs*save_rate and loss.item()<best:
                    best = loss.item()
                    torch.save(model.state_dict(), f'./model_fold_{fold + 1}.ckpt')
#                    print(f"loss:{loss.item()} saving...")
                loss.backward()
                optimizer.step()

            losses.append(loss.item())
            lr_scheduler.step()
#            if (epoch+1) % 100 == 0 :
#                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.10f}')
#            bar.set_postfix({ 'Loss': f'{loss.item():.10f}','lr':f"{lr_scheduler.get_lr()[0]:.6f}"})

        model.load_state_dict(torch.load(f'./model_fold_{fold + 1}.ckpt'))
        model.eval()
        
        with torch.no_grad():
            _,test_outputs = model(X_test_tensor)
            y_pred = test_outputs.to('cpu')
            test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
            rmse = torch.sqrt(test_loss)  # 计算均方根误差
            y_out = (y_pred - y_test_tensor.numpy().reshape(len(y_test_tensor),1))/y_test_tensor.numpy().reshape(len(y_test_tensor),1)
            y_out = y_out.abs() < 0.1
            accuracy = y_out.sum()/len(y_out)
#            print(f'Test Loss: {test_loss:.10f}, Test RMSE: {rmse:.10f},Accuracy: {accuracy:.10f}')
            
        X_test = scaler.inverse_transform(X_test)
        fold_df = pd.DataFrame(X_test, columns = [str(i) for i in range(1, X[0].shape[0]+1)])
        fold_df['Test Labels'] = y_test
        fold_df['Test Outputs'] = y_pred.numpy()
        fold_df['Fold'] = fold + 1
        results_df = pd.concat([results_df, fold_df], ignore_index=True)
        test_losses.append(test_loss*len(y_test))
        
    print(f'The {num_while} times of test: The RMSE of this test:{torch.sqrt(sum(test_losses)/len(y)):.10f} and Linear_size1 = {num1} and Linear_size2 = {num2}')

    
    num2-=1
    if num2 == 11:
        num2 = 20
    if num2 == 20:
        num1 -=1
    if num1 == 5:
        num1 = 10
    
results_df.to_excel('test_data_and_outputs.xlsx', index=False)
print(f'Times of test:{num_while} and Linear_size1 = {num1} and Linear_size2 = {num2}')

The 1 times of test: The RMSE of this test:0.4289597571 and Linear_size1 = 8 and Linear_size2 = 16
The 2 times of test: The RMSE of this test:0.5101928711 and Linear_size1 = 8 and Linear_size2 = 15
The 3 times of test: The RMSE of this test:0.4134084880 and Linear_size1 = 8 and Linear_size2 = 14
The 4 times of test: The RMSE of this test:0.4982110858 and Linear_size1 = 8 and Linear_size2 = 13
The 5 times of test: The RMSE of this test:0.5408782959 and Linear_size1 = 8 and Linear_size2 = 12
The 6 times of test: The RMSE of this test:0.4933682680 and Linear_size1 = 7 and Linear_size2 = 20
The 7 times of test: The RMSE of this test:0.4372816682 and Linear_size1 = 7 and Linear_size2 = 19
The 8 times of test: The RMSE of this test:0.4336703718 and Linear_size1 = 7 and Linear_size2 = 18
The 9 times of test: The RMSE of this test:0.6164389253 and Linear_size1 = 7 and Linear_size2 = 17
The 10 times of test: The RMSE of this test:0.4301291406 and Linear_size1 = 7 and Linear_size2 = 16
The 11 ti